In [879]:
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
sys.path.append(os.path.join("..","Libraries","QML_lib"))
import Evo as evo

global paulis_list
paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}

import hashlib

# Naming Convention
### Definitions

In [642]:
class operator():
    # Better to have this info in a DB than recalculating when called?
    def __init__(self, name): 
        self.name = name
    
    @property
    def constituents_names(self):
        t_str, p_str, max_t, max_p = get_t_p_strings(self.name)
        paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}
        if(max_t >= max_p):
            #constituent_names.append(self.name)
            return [self.name]
        else: 
            return self.name.split(p_str)
            #constituent_names = self.name.split(p_str)
        #return constituent_names        

    @property
    def num_qubits(self):
        return get_num_qubits(self.name)
        
    @property
    def constituents_operators(self):
        ops = []
        for i in self.constituents_names:
            ops.append(compute(i))
        return ops

    @property
    def num_constituents(self):
        # 1 param per constituent?
        return len(self.constituents_names)
    
    @property 
    def matrix(self):
        mtx = empty_array_of_same_dim(self.name)
        #dim = 2**self.num_qubits
        #print("Constructing matrix of dim: ", dim)
        #mtx = np.zeros([dim, dim], np.complex128)
        for i in self.constituents_operators:
            mtx += i
        return mtx
    @property
    def qubits_acted_on(self):
        return list_used_qubits(self.name)
   
    @property 
    def two_to_power_used_qubits_sum(self):
        running_sum = 0
        for x in list_used_qubits(self.name):
            running_sum += 2**x
        return running_sum

    @property
    def alph_name(self):
        return alph(self.name)
    
    
    def test_const(self):
        for i in self.constituents_operators:
            print(i)
            
    
    def interactions_on(self):
        # work out which qubits have actions; list ?
        return 0

def get_num_qubits(name):
    max_t_found = 0 
    t_str=''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    num_qubits = len(t_str) + 1
    return num_qubits
    

def list_used_qubits(name):
    max_t, t_str = find_max_letter(name, "T")
    max_p, p_str = find_max_letter(name, "P")
    running_list = []

    if max_p >= max_t:
        list_by_p_sep = []
        sep_by_p = name.split(p_str)
        for x in sep_by_p:
            list_by_p_sep.append(get_acted_on_qubits(x))

        for i in range(len(list_by_p_sep)):
            to_add= list(set(list_by_p_sep[i]) - set(running_list))
            running_list = running_list + to_add

    else:
        running_list = get_acted_on_qubits(name)
    return running_list


def get_acted_on_qubits(name):
    max_t, t_str = find_max_letter(name, "T")
    max_p, p_str = find_max_letter(name, "P")
    if max_p > max_t:
        list_by_p_sep = []
        sep_by_p = name.split(p_str)
        for x in sep_by_p:
            list_by_sep.append(fill_qubits_acted_on_list, x)
    
    
    qubits_acted_on = []
    fill_qubits_acted_on_list(qubits_acted_on,name)
    return sorted(qubits_acted_on)
    
def fill_qubits_acted_on_list(qubits_acted_on, name):
    max_t, t_str = find_max_letter(name, "T")
    max_p, p_str = find_max_letter(name, "P")
    if(max_p > max_t):
        string_to_analyse = name.split(p_str)[0]
    else:
        string_to_analyse = name

    if max_t == 0:
        if string_to_analyse != 'i':
            qubits_acted_on.append(1)


    else:
        i=max_t
        this_t_str = t_str
        broken_down = string_to_analyse.split(this_t_str)
        lhs = broken_down[0]
        rhs = broken_down[1]
        if rhs !='i':
            qubits_acted_on.append(i+1)

        if max_t == 1:
            if lhs!='i':
                qubits_acted_on.append(1)
        else: 
            fill_qubits_acted_on_list(qubits_acted_on, lhs)                
    
def get_t_p_strings(name):
    t_str = ''
    p_str = ''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    while name.count(p_str+'P')>0:
        p_str=p_str+'P'

    max_t = len(t_str)
    max_p = len(p_str)

    return t_str, p_str, max_t, max_p        
    
def find_max_letter(string, letter):
    letter_str=''
    while string.count(letter_str+letter)>0:
        letter_str=letter_str+letter

    return len(letter_str), letter_str


def empty_array_of_same_dim(name):
    t_str=''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    num_qubits = len(t_str) +1
    dim = 2**num_qubits
    #print("String: ", name, " has NQubits: ", num_qubits)
    empty_mtx = np.zeros([dim, dim], dtype=np.complex128)
    return empty_mtx



def alph(name):
    t_max, t_str = find_max_letter(name, "T")
    p_max, p_str = find_max_letter(name, "P")
    m_max, m_str = find_max_letter(name, "M")
    
    if p_max == 0 and t_max ==0 and p_max ==0 :
        return name
    
    if p_max > t_max and p_max > m_max: 
        ltr = 'P'
        string = p_str
    elif t_max >= p_max:
        string = t_str
        ltr = 'T'
    elif m_max >= p_max: 
        string = m_str
        ltr = 'M'
    elif t_max > m_max: 
        string = t_str
        ltr = 'T'
    else:
        ltr = 'M'
        string = m_str

    spread = name.split(string)
    if  p_max==m_max and p_max > t_max:
        string = p_str
        list_elements = name.split(p_str)
        
        for i in range(len(list_elements)):
            list_elements[i] = alph(list_elements[i])
        sorted_list = sorted(list_elements)
        linked_sorted_list = p_str.join(sorted_list)
        return linked_sorted_list
        
    if ltr=='P' and p_max==1:
        sorted_spread = sorted(spread)
        out = string.join(sorted_spread)
        return out
    elif ltr=='P' and p_max>1:
        list_elements = name.split(string)
        sorted_list = sorted(list_elements)
        for i in range(len(sorted_list)):
            sorted_list[i] = alph(sorted_list[i])
        linked_sorted_list = string.join(sorted_list)
        return linked_sorted_list
    else: 
        for i in range(len(spread)):
            spread[i] = alph(spread[i])
        out = string.join(spread)
        return out


def compute_t(inp):
   # print("Compute t : ", inp)
    max_t, t_str = find_max_letter(inp, "T")
    max_p, p_str = find_max_letter(inp, "P")

    if(max_p == 0 and max_t==0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 

    elif(max_t==0):
        return compute(inp)
    else:
        to_tens = inp.split(t_str)
        #print("To tens: ", to_tens)
        running_tens_prod=compute(to_tens[0])
        #print("Split by ", t_str, " : \n", to_tens)
        for i in range(1,len(to_tens)):
            max_p, p_str = find_max_letter(to_tens[i], "P")
            max_t, t_str = find_max_letter(to_tens[i], "T")
            #print("To tens [i=", i, "]:\n", to_tens[i] )
            rhs = compute(to_tens[i])
            running_tens_prod = np.kron(running_tens_prod, rhs)
        #print("RESULT ", t_str, " : ", inp, ": \n", running_tens_prod)
        return running_tens_prod

def compute_p(inp):
    #print("Compute p : ", inp)
    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")

    if(max_p == 0 and max_t==0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 

    elif max_p==0:
        return compute(inp)
    else: 
        to_add = inp.split(p_str)
        #print("To add : ", to_add)
        running_sum = empty_array_of_same_dim(to_add[0])
        for i in range(len(to_add)):
            max_p, p_str = find_max_letter(to_add[i], "P")
            max_t, t_str = find_max_letter(to_add[i], "T")

           # print("To add [i=", i, "]:", to_add[i] )
            rhs = compute(to_add[i])
            #print("SUM shape:", np.shape(running_sum))
            #print("RHS shape:", np.shape(rhs))
            running_sum += rhs

        #print("RESULT ", p_str, " : ", inp, ": \n", running_sum)
        return running_sum


def compute_m(inp):
    max_m, m_str = find_max_letter(inp, "M")
    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")

    if(max_m == 0 and max_t==0 and max_p == 0 ):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 

    elif max_m ==0:
        return compute(inp)
    
    else:   
        to_mult = inp.split(m_str)
        #print("To mult : ", to_mult)
        t_str=''
        while inp.count(t_str+'T')>0:
            t_str=t_str+'T'

        num_qubits = len(t_str) +1
        dim = 2**num_qubits

        running_product = np.eye(dim)

        for i in range(len(to_mult)):
            #print("Running product : \n", running_product, "\n times \n", compute(to_mult[i]))
            running_product = np.dot(running_product, compute(to_mult[i]))

        return running_product    
    
def compute(inp):
    #print("Computing ", inp)
    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")
    max_m, m_str = find_max_letter(inp, "M")

    if(max_m == 0 and max_t==0 and max_p == 0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 
    elif max_m > max_t:
        return compute_m(inp)
    elif max_t >= max_p:
       # print("Max t=", max_t, ">= max p=", max_p)
        return compute_t(inp)
    else:
        return compute_p(inp)    


In [575]:
name='yPx'
op = operator(name)
print(op.alph_name)

xPy


'xTy'

In [312]:
running_sum = 0 
for i in ls:
    print("i=", i, ": ", 2**i)
    running_sum += 2**i
    print("now = ", running_sum)
print(running_sum)     

i= 1 :  2
now =  2
i= 2 :  4
now =  6
6


In [350]:
def sortmebaby(thestring):
    splitted = thestring.split("T")
    allsorted = [ "P".join(sorted(item.split("P")) ) for item in splitted    ]
    thestring = "T".join(allsorted)
     
    return thestring

In [363]:
sortmebaby('yPxTzPPzTx')

'xPyTPzPzTx'

In [358]:
name='iTyTTTiTx'
splitted = name.split("T")
allsorted = [ "P".join(sorted(item.split("P")) ) for item in splitted    ]
thestring = "T".join(allsorted)
   

In [401]:
def sort_at_p_level(name):
    print("To sort: ", name)
    max_p, p_str = find_max_letter(name, "P")
    if max_p > 0:
        split_by_p = name.split(p_str)
        if max_p > 1: 
            for i in range(len(split_by_p)):
                split_by_p[i] = sort_at_p_level(split_by_p[i])
        #print(split_by_p)
        sorted_around_p = sorted(split_by_p)
        sorted_around_p_str = p_str.join(sorted_around_p)
        print("Sorted : ", sorted_around_p_str)
        return sorted_around_p_str
    else: 
        return name

In [406]:
name='yPx'
t_max, t_str = find_max_letter(name, "T")
p_max, p_str = find_max_letter(name, "P")
m_max, m_str = find_max_letter(name, "M")

if p_max >0:
    split_p = name.split(p_str)
else: 
    split_p = name
print(split_p)

['y', 'x']


In [453]:
def alph_by_str(name, letter):
    print("alph by string: ", name, "letter ", letter)
    max_letter, letter_str = find_max_letter(name, letter)
    
    if(max_letter > 0):
        broken_down = name.split(letter_str)
        alphabetised = sorted(broken_down)
        rejoined = letter_str.join(alphabetised)
        print("sorted:", rejoined)
        return rejoined
    else:
        print("Max letter =0", max_letter, letter_str)
        return name

In [ ]:
def sort_

In [562]:
def alph(name):
    #print("Alph: ", name)
    t_max, t_str = find_max_letter(name, "T")
    p_max, p_str = find_max_letter(name, "P")
    m_max, m_str = find_max_letter(name, "M")
    
    if p_max == 0 and t_max ==0 and p_max ==0 :
        return name
    
    if p_max > t_max and p_max > m_max: 
        ltr = 'P'
        string = p_str
    elif t_max >= p_max:
        string = t_str
        ltr = 'T'
    elif m_max >= p_max: 
        string = m_str
        ltr = 'M'
    elif t_max > m_max: 
        string = t_str
        ltr = 'T'
    else:
        ltr = 'M'
        string = m_str

    #print("letter ", ltr, " string ", string)
    spread = name.split(string)
    #print("spread : ", spread)
    if  p_max==m_max and p_max > t_max:
        string = p_str
        list_elements = name.split(p_str)
        print("list", list_elements)
        
        for i in range(len(list_elements)):
            list_elements[i] = alph(list_elements[i])
        sorted_list = sorted(list_elements)
        linked_sorted_list = p_str.join(sorted_list)
        return linked_sorted_list
        
    if ltr=='P' and p_max==1:
        sorted_spread = sorted(spread)
        out = string.join(sorted_spread)
        return out
    elif ltr=='P' and p_max>1:
        list_elements = name.split(string)
        sorted_list = sorted(list_elements)
        for i in range(len(sorted_list)):
            sorted_list[i] = alph(sorted_list[i])
        linked_sorted_list = string.join(sorted_list)
        return linked_sorted_list
    else: 
        print("String ", string, "Spread: ", spread)
        for i in range(len(spread)):
            #print("Spread [i=", i, "] = ", spread[i])
            spread[i] = alph(spread[i])
            #print("After fnc: spread[i]=", spread[i])
        out = string.join(spread)
        return out

In [868]:
test_name = 'xPyPzPiTxPyPzPiPxPyPzPiMMxPyPzPiTxPyPzPiPxPyPzPi'
out=alph(test_name)

print("In: " ,test_name)
print("Out: ", out)

In:  xPyPzPiTxPyPzPiPxPyPzPiMMxPyPzPiTxPyPzPiPxPyPzPi
Out:  iPxPyPzTiPiPxPxPyPyPzPzMMiPxPyPzTiPiPxPxPyPyPzPz


In [429]:
name = 'yPxTzPi'
t_max, t_str = find_max_letter(name, "T")
p_max, p_str = find_max_letter(name, "P")
m_max, m_str = find_max_letter(name, "M")

if m_max == 0:
    m_str='M'
if t_max == 0:
    t_str ='T'
if p_max == 0:
    p_str='P'

split_t = name.split(t_str)
for i in range(len(split_t)):
    split_t[i] = alph_list_by_str(split_t[i], p_str)
    
sorted_about_p=t_str.join(split_t)
sorted_about_t = alph_list_by_str(sorted_about_p, t_str)

In [430]:
sorted_about_t

'iPzTxPy'

In [421]:
name='yPxPzTiPx'
print(alph_list_by_str(name, 'T'))

iPxTyPxPz


In [402]:
name = 'yPxTzPPzTx'

out= sort_at_p_level(name)

print("input: ", name)
print("output: ", out)

To sort:  yPxTzPPzTx
To sort:  yPxTz
Sorted :  xTzPy
To sort:  zTx
Sorted :  xTzPyPPzTx
input:  yPxTzPPzTx
output:  xTzPyPPzTx


In [370]:
t_max, t_str = find_max_letter(name, "T")
p_max, p_str = find_max_letter(name, "P")
m_max, m_str = find_max_letter(name, "M")

In [391]:
sort_at_p_level('yPxPzPiPPxPi')

['y', 'x', 'z', 'i']
['x', 'i']
['iPxPyPz', 'iPx']


'iPxPPiPxPyPz'

array([[ 0.+0.j,  0.-1.j],
       [ 0.+1.j,  0.+0.j]])

In [349]:
a.qubits_acted_on

[2, 4, 1]

In [338]:
my_string = 'hello'

In [340]:
'b(my_string)'

'b(my_string)'

# Database

In [937]:
def launch_db(RootN_Qbit=[0], N_Qubits=1, gen_list=[]):
    
    # Create DB using strings of names
    generators = []
    total_model_list = []
    Max_N_Qubits = 11
    model_lists = {}
    for j in range(1, Max_N_Qubits):
        model_lists[j] = []
    
    for i in gen_list:
        generators.append(operator(i))
        alph_model_name = alph(i)
        num_qubits = get_num_qubits(i)
        model_lists[num_qubits].append(alph_model_name)

    model_db = pd.DataFrame({
        '<Name>' : [ gen.name for gen in generators], 
        'Alph_Name' :[ gen.alph_name for gen in generators],
        'All_Operators_Names': [gen.constituents_names for gen in generators],
        'All_Operators_Matrices': [gen.constituents_operators for gen in generators],
        'N_params' : [ gen.num_constituents for gen in generators ],
        'Num_Qubits' : [ gen.num_qubits for gen in generators ],
        'Qubits_Acted_On' : [ gen.qubits_acted_on for gen in generators ],
        'Binary_Sum_Used_Qubits' : [ gen.two_to_power_used_qubits_sum for gen in generators ],
        'Matrix' : [gen.matrix for gen in generators]
    })
        
    # if N_qubits defined: work out generator list.
    # Or should number qubits be implied by gen list?
    db = pd.DataFrame({
        '<Name>' : [ gen.name for gen in generators], 
        'Alph_Name' :[ gen.alph_name for gen in generators],
        'N_Qbit' : [RootN_Qbit for i in range(len(gen_list))],
        'DB_location' : [ get_location(model_db, gen.name) for gen in generators],
        'Status' : 'Ready', 
        'Selected' : False, 
        'LogL_Ext' : None, 
        'QML_Class' : None, 
        'Origin_epoch' : 0, 
        'RootNode' : 'NaN',
        })  
        
    return db, model_db, model_lists

def get_location(db, name):
    for i in range(len(db['<Name>'])):
        if db['<Name>'][i] == name:
            return i

def get_location_by_alph_name(db, name):
    for i in range(len(db['Alph_Name'])):
        if db['Alph_Name'][i] == name:
            return i

        
def consider_new_model(model_lists, name, db):
    # Return true indicates it has not been considered and so can be added
    al_name = alph(name)
    n_qub = get_num_qubits(name)
    if al_name not in model_lists[n_qub]:
        return 'New'
    else: 
        location = get_location_by_alph_name(db, al_name)
        return location
        
def add_model(model_name, running_database , model_db = model_db, model_lists = model_lists):
    ## Alternatively, can use existing model_db and total_model_list
    ## What is preferred?
    alph_model_name = alph(model_name)
    model_num_qubits = get_num_qubits(model_name)
    
    if consider_new_model(model_lists, model_name, running_database)== 'New':
        print("Model Not previously considered -- adding")
        op = operator(model_name)
        num_rows = len(running_database)

        # add model_db_new_row to model_db and running_database
        # Note: do NOT use pd.df.append() as this copies total DB,
        # appends and returns copy.
        
        model_db_new_row = pd.Series({
            '<Name>': op.name, 
            'Alph_Name': op.alph_name,
            'All_Operators_Names' : op.constituents_names,
            'All_Operators_Matrices': op.constituents_operators,
            'N_params' : op.num_constituents,
            'Num_Qubits' : op.num_qubits,
            'Qubits_Acted_On' : op.qubits_acted_on,
            'Binary_Sum_Used_Qubits' : op.two_to_power_used_qubits_sum,
            'Matrix' : op.matrix
            })

        model_db.loc[num_rows] = model_db_new_row
       # total_model_list.append(alph_model_name)
        model_lists[model_num_qubits].append(alph_model_name)
        
        # Add to running_database too, after adding to model_db
        
        running_db_new_row = pd.Series({
            '<Name>': op.name,
            'Alph_Name' : op.alph_name,
            'N_Qbit' : None, #TODO TODO
            'DB_location' : get_location(model_db, model_name),
            'Status' : 'Ready', 
            'Selected' : False, 
            'LogL_Ext' : None, 
            'QML_Class' : None, 
            'Origin_epoch' : 0, 
            'RootNode' : 'NaN',
        })

        running_database.loc[num_rows] = running_db_new_row      
        
    else:
        location = consider_new_model(model_lists, model_name, running_database)
        #db_loc = get_location_by_alph_name(model_db, model_name)
        print("Model", alph_model_name, " previously considered at location", location)  

## Example usage of database

In [938]:
alph('xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi')

'xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi'

In [939]:
name1 = 'yTxPPzTi'
name2 = 'xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi'
name3 = 'xTy'

gen_list=[name1, name2, name3]
db, model_db, model_lists = launch_db(gen_list=gen_list)

In [940]:
db

,<Name>,Alph_Name,DB_location,LogL_Ext,N_Qbit,Origin_epoch,QML_Class,RootNode,Selected,Status
0,yTxPPzTi,yTxPPzTi,0,None,[0],0,None,NaN,False,Ready
1,xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,1,None,[0],0,None,NaN,False,Ready
2,xTy,xTy,2,None,[0],0,None,NaN,False,Ready


In [943]:
add_model('yTiPPxTyTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi', db, model_db, model_lists)

Model xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi  previously considered at location 1


In [915]:
model_lists

{1: [],
 2: ['yTxPPzTi', 'xTy'],
 3: ['xPyTzTTi'],
 4: ['xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi',
  'xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi',
  'xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi'],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: []}

In [913]:
model_db

,<Name>,All_Operators_Matrices,All_Operators_Names,Alph_Name,Binary_Sum_Used_Qubits,Matrix,N_params,Num_Qubits,Qubits_Acted_On
0,yTxPPzTi,"[[[0j, 0j, 0j, -1j], [0j, 0j, -1j, 0j], [0j, 1...","[yTx, zTi]",yTxPPzTi,6,"[[(1+0j), 0j, 0j, -1j], [0j, (1+0j), -1j, 0j],...",2,2,"[1, 2]"
1,xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,"[[[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-1-0j)...","[xTyPPyTiTTTiTyPPxTi, xTyPPyTiTTTiTyPPxTi]",xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,22,"[[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-2+0j),...",2,4,"[1, 2, 4]"
2,xTy,"[[[0j, 0j, 0j, -1j], [0j, 0j, 1j, 0j], [0j, -1...",[xTy],xTy,6,"[[0j, 0j, 0j, -1j], [0j, 0j, 1j, 0j], [0j, -1j...",1,2,"[1, 2]"
3,yPxTzTTi,"[[[0j, 0j, 0j, 0j, (1-1j), 0j, 0j, 0j], [0j, 0...",[yPxTzTTi],xPyTzTTi,6,"[[0j, 0j, 0j, 0j, (1-1j), 0j, 0j, 0j], [0j, 0j...",1,3,"[1, 2]"
4,xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,"[[[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-1-0j)...","[xTyPPyTiTTTiTyPPxTi, xTyPPyTiTTTiTyPPxTi]",xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,22,"[[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-2+0j),...",2,4,"[1, 2, 4]"



# Old Development code 

### Initialise a database by passing a list of generator names (strings)

In [873]:
import hashlib

In [944]:
a = 'xTy'
a_hash = hashlib.md5(a.encode())
b = 'xTy'
b_hash = hashlib.md5(b.encode())

In [945]:
a_hash==b_hash

False

In [877]:
my_string = 'xTz'
#test_list = []


test_hash=hashlib.md5(my_string.encode())
test_list.append(test_hash)

In [878]:
test_list

[<md5 HASH object @ 0x7f2a5b68cb20>, <md5 HASH object @ 0x7f2a5b66aad0>]

In [3]:
def Init_N_Qubit_DB(RootN_Qbit=[0], N_Qubits=1, gen_list=[]):
    
    # Create DB using strings of names
    model_db, total_model_list = launch_model_db(gen_list)    
    generators = []
   
    for i in gen_list:
        generators.append(operator(i))
    # if N_qubits defined: work out generator list.
    # Or should number qubits be implied by gen list?
    db = pd.DataFrame({
        '<Name>' : [ [gen.name] for gen in generators], 
        'N_Qbit' : [RootN_Qbit for i in range(len(generators))],
        'N_params' : [ gen.num_constituents for gen in generators ],
        'Num_Qubits' : [ gen.num_qubits for gen in generators ],
        'DB_location' : [ get_location(model_db, gen.name) for gen in generators],
        'Status' : 'Ready', 
        'Selected' : False, 
        'LogL_Ext' : None, 
        'QML_Class' : None, 
        'Origin_epoch' : 0, 
        'RootNode' : 'NaN',
        'All_Operators_Names': [gen.constituents_names for gen in generators],
        'All_Operators_Matrices': [gen.constituents_operators for gen in generators]
        })  
        
    return db

def get_location(db, name):
    for i in range(len(db['<Name>'])):
        if db['<Name>'][i] == [name]:
            return i

In [138]:
def launch_model_db_standalone(model_list):
    # List of operatoer class instances
    operator_list = []
    # List of model names (strings)
    total_model_list = []
    for model in model_list:
        operator_list.append(operator(model))
        total_model_list.append(model)
   
    model_db = pd.DataFrame({
        '<Name>' : [   [gen.name] for gen in operator_list], 
        'All_Operators_Names': [gen.constituents_names for gen in operator_list],
        'All_Operators_Matrices': [gen.constituents_operators for gen in operator_list],
        'Matrix' : [gen.matrix for gen in operator_list]
    })
    
    return model_db, total_model_list

def add_model_to_db_standalone(model_name, model_db, total_model_list):
    ## Alternatively, can use existing model_db and total_model_list
    ## What is preferred?
    
    if model_name not in total_model_list:
        print("Model Not previously considered -- adding")
        op = operator(model_name)
        this_row = pd.Series({
            '<Name>':op.name, 
            'All_Operators_Names' : op.constituents_names,
            'All_Operators_Matrices': op.constituents_operators,
            'Matrix' : op.matrix
            })

        num_rows = len(total_model_list)
        # add this_row to model_db
        # Note: do NOT use pd.df.append() as this copies total DB,
        # appends and returns copy.
        model_db.loc[num_rows] = this_row
        total_model_list.append(model_name)
    else: 
        print("Model previously considered")

In [274]:
### Development to include multiplication

In [276]:
name1 = 'xTyTTzMxTTTiMMMMxTyTTzTTTi'
print(np.shape(compute(name1)))

To mult :  ['xTyTTzMxTTTi', 'xTyTTzTTTi']
To tens:  ['xTyTTzMx', 'i']
To tens:  ['xTy', 'zMx']
To tens:  ['x', 'y']
To mult :  ['z', 'x']
To tens:  ['xTyTTz', 'i']
To tens:  ['xTy', 'z']
To tens:  ['x', 'y']
(16, 16)


In [250]:
name = 'xMyMzMiPiMxTy'
print(compute(name))

[[ 0.+0.j  1.+0.j  0.+0.j  0.-1.j]
 [-1.+0.j  0.+0.j  0.+1.j  0.+0.j]
 [ 0.+0.j  0.-1.j  0.+0.j  1.+0.j]
 [ 0.+1.j  0.+0.j -1.+0.j  0.+0.j]]


In [198]:
np.dot(x,y)

array([[ 0.+1.j,  0.+0.j],
       [ 0.+0.j,  0.-1.j]])

In [253]:
new_name = 'xPyMxPzTxPyPzTTxMyPyPzMxTTTxPyPz'
name2 = 'xPyMxPzTxPyPzTTxMyPyPzMxTTTxPyPy'


In [256]:
com1 = compute(new_name)

In [22]:
a = operator('xTiTTiTTTiPPPPiTxTTiTTTi')

In [28]:
np.shape(a.constituents_operators[0])

(16, 16)

In [29]:
np.shape(a.matrix)

(16, 16)

In [273]:
a='xMzPyTzTTyTTTz'
print(np.shape(compute(a)))

(16, 16)


In [21]:
def merge(s1, s2):
    max_t_1, t_str_1 = find_max_letter(s1, "T")
    max_t_2, t_str_2 = find_max_letter(s2, "T")    

    p_str = ''
    for i in range(len(t_str_1)):
        p_str = p_str+'P'
    
    print(p_str)
    
    split_1 = s1.split(t_str_1)
    split_2 = s2.split(t_str_2)
    
    print("split 1", split_1)
    print("split 2", split_2)
    out_str=[]
    
    out=''
    for i in range(len(split_1)):
        if i > 0 :
            out += t_str_1
        print("i=", i, split_1[i], split_2[i])
        out = out+ split_1[i] +p_str + split_2[i]
    return out


In [28]:
s1='xPiTyPz'
s2='zTi'

In [29]:
merge(s1,s2)

P
split 1 ['xPi', 'yPz']
split 2 ['z', 'i']
i= 0 xPi z
i= 1 yPz i


'xPiPzTyPzPi'

In [258]:
find_acted_on('xTyPxTTiPPPzTiTTi')

[1, 2]

In [238]:
running_list=[]
running_list = list_by_p_sep[0]

In [232]:
running_list

[1, 2]

In [216]:
p_qubits_list

[2, 1]

In [205]:
name = 'yTiTTxTTTz'
a=get_acted_on_qubits(name)
print(a)

Name:  yTiTTxTTTz
String to analyse:  yTiTTxTTTz
t_str:  TTT
['yTiTTx', 'z']
Adding qubit 4 because rhs= z
Name:  yTiTTx
String to analyse:  yTiTTx
t_str:  TT
['yTi', 'x']
Adding qubit 3 because rhs= x
Name:  yTi
String to analyse:  yTi
t_str:  T
['y', 'i']
Not adding qubit 2 because rhs =  i
Adding qubit  1 because lhs =  y
[4, 3, 1]
[1, 3, 4]


In [206]:
a

[1, 3, 4]

In [187]:
a = get_acted_on_qubits('x')

Name:  x
Adding qubit 1


In [188]:
a

[]

In [62]:
this_t_str=''
for i in range(max_t):
    this_t_str=this_t_str+'T'
    print(this_t_str)


T
TT


In [38]:
acted_on = []


for i in range(len(split_by_t)):
    if split_by_t[i]!='i':
        acted_on.append(i)

        

In [39]:
acted_on

[0]

In [32]:
max_t, t_str = find_max_letter(name, "T")

check_str = ''
for i in range(max_t):
    check_str += "T"
    split_list = 

In [92]:
import locale
def reorder(name):
    max_t, t_str = find_max_letter(name, "T")
    max_p, p_str = find_max_letter(name, "P")
    if max_p==1:
        to_add = name.split(p_str)
        sorted_to_add = sorted(to_add)
        return p_str.join(sorted_to_add)
    else:
        return name
    

In [94]:
reorder('yPx')

'xPy'

In [107]:
def split_letter(name, letter):
    max_letter, letter_str = find_max_letter(name, letter)
    if max_letter > 0:
        return name.split(letter_str)
    else: 
        return [name]
    

In [108]:
name  = 'yPxTz'    

In [119]:

split_t  = split_letter(name, "T")
p_list = []
m_str='M'
for t_sub in split_t:
    split_m = split_letter(t_sub, "M")
    if len(split_m) >0:
        m_list = []
    for m_sub in split_m:
        split_p = split_letter(m_sub, "P")
        rearranged_by_p = sorted(split_p)
        p_str='P'
        
        p_list.append(p_str.join(rearranged_by_p))
    if len(split_m) >0:
        m_list.append(m_str.join(p_list))
    else: 
        m_list =(p_list)
out_string = t_str.join(m_list)

print(out_string)


xPyMz


In [120]:
split_t('yPxTz')

TypeError: 'list' object is not callable

In [81]:
ls = ['x', 'z','y']

In [74]:
print(sorted(ls))

['x', 'y', 'z']


In [89]:
def do_commute(a,b):
    ab = np.dot(a,b)
    ba = np.dot(b,a)
    diff = ab - ba
    if np.max(np.abs(diff)) != 0:
        print("Do not commute: ")
        print("Diff: \n", diff)
    else:
        print("Commute")
    

In [90]:
x = evo.sigmax()
y = evo.sigmay()
z = evo.sigmaz()
i = np.eye(2)

In [91]:
do_commute(x,y)

Do not commute: 
Diff: 
 [[ 0.+2.j  0.+0.j]
 [ 0.+0.j  0.-2.j]]
